In [33]:
import numpy as np
from mlxtend.evaluate import cochrans_q
from mlxtend.evaluate import mcnemar_table
from mlxtend.evaluate import mcnemar
from results import *

Cochran's Q-test

In [34]:
list_of_values = ["common", "justice", "virtue", "dentology", "all_categories"]

for value in list_of_values:
    y_model_1 = globals()["gpt35_" + value]

    y_model_2 = globals()["gpt4_" + value]

    y_model_3 = globals()["llama70_" + value]

    y_model_4 = globals()["Gemini_" + value]

    y_model_5 = globals()["Claude3_" + value]

    y_true = globals()["y_" + value]

    q, p_value = cochrans_q(y_true, 
                        y_model_1, 
                        y_model_2, 
                        y_model_3,
                        y_model_4,
                        y_model_5
                        )
    print (value)
    print('Q:', q)
    print('p-value:', p_value)
    print()


common
Q: 84.10526315789474
p-value: 2.348409661759755e-17

justice
Q: 50.20289855072464
p-value: 3.275244409497549e-10

virtue
Q: 13.26530612244898
p-value: 0.010049649085819049

dentology
Q: 39.54140127388535
p-value: 5.3844947321518655e-08

all_categories
Q: 124.43392504930966
p-value: 6.030455535286086e-26



mcnemar test

In [35]:
import scipy.stats
import scipy.stats as st

def mcnemar(y_true, yhatA, yhatB, alpha=0.05):
    """
    Perform McNemar's test to compare the accuracy of two classifiers.

    Parameters:
    - y_true: array-like, true labels
    - yhatA: array-like, predicted labels by classifier A
    - yhatB: array-like, predicted labels by classifier B
    - alpha: float, significance level (default: 0.05)

    Returns:
    - thetahat: float, estimated difference in accuracy between classifiers A and B
    - CI: tuple, confidence interval of the estimated difference in accuracy
    - p: float, p-value for the two-sided test of whether classifiers A and B have the same accuracy
    """

    nn = np.zeros((2, 2))
    c1 = yhatA - y_true == 0
    c2 = yhatB - y_true == 0

    nn[0, 0] = sum(c1 & c2)
    nn[0, 1] = sum(c1 & ~c2)
    nn[1, 0] = sum(~c1 & c2)
    nn[1, 1] = sum(~c1 & ~c2)

    n = sum(nn.flat)
    n12 = nn[0, 1]
    n21 = nn[1, 0]

    thetahat = (n12 - n21) / n
    Etheta = thetahat

    Q = (
        n**2
        * (n + 1)
        * (Etheta + 1)
        * (1 - Etheta)
        / ((n * (n12 + n21) - (n12 - n21) ** 2))
    )

    p = (Etheta + 1) * 0.5 * (Q - 1)
    q = (1 - Etheta) * 0.5 * (Q - 1)

    CI = tuple(lm * 2 - 1 for lm in scipy.stats.beta.interval(1 - alpha, a=p, b=q))

    p = 2 * scipy.stats.binom.cdf(min([n12, n21]), n=n12 + n21, p=0.5)

    return nn, thetahat, CI, p

list_of_values = ["common", "justice", "virtue", "dentology", "all_categories"]
category = ''

count = 0

for value in list_of_values:
    y_model_1 = globals()["gpt35_" + value]

    y_model_2 = globals()["gpt4_" + value]

    y_model_3 = globals()["llama70_" + value]

    y_model_4 = globals()["Gemini_" + value]

    y_model_5 = globals()["Claude3_" + value]

    y_true = globals()["y_" + value]

    models = [y_model_1, y_model_2, y_model_3, y_model_4, y_model_5]

    for i in range(len(models)):
        for j in range(i + 1, len(models)):
            model1 = models[i]
            model2 = models[j]

            nn, thetahat, CI, p = mcnemar(y_true, model1, model2)
            
            model1name = ''
            model2name = ''
            if(i == 0):
                model1name = "GPT-3.5"
            elif(i==1):
                model1name = "GPT-4o"
            elif(i==2):
                model1name = "Llama"
            elif(i==3):
                model1name = "Gemini"
            elif(i==4):
                model1name = "Claude3"
            
            if(j == 0):
                model2name = "GPT-3.5"
            elif(j==1):
                model2name = "GPT-4o"
            elif(j==2):
                model2name = "Llama"
            elif(j==3):
                model2name = "Gemini"
            elif(j==4):
                model2name = "Claude3"
            
            if(category != value):
                category = value
                print('----------------- ' + category + ' -----------------')
            
            alpha = 0.05
            n = 40
            
            significance = p < alpha/n

            if(significance):
                count += 1

            print(model1name,model2name, p, '------',significance)

print(count)

# 95% sandsynlighed for at der er signifkant forskel.
# 5% af tiden har vi taget fejl, vi siger de er forskellige men de er ens.
# 
# 1 gang: sandsynligheden for fejlen er 5%
# 5 gang: sandsynligheden for fejlen er 25% (gæt)
# 40 gange: sandsynligheden er meget stor
# derfor gør vi alpha mindfre 
            
            


----------------- common -----------------
GPT-3.5 GPT-4o 1.862645149230957e-09 ------ True
GPT-3.5 Llama 0.6900379657745361 ------ False
GPT-3.5 Gemini 2.384185791015625e-07 ------ True
GPT-3.5 Claude3 2.9802322387695312e-08 ------ True
GPT-4o Llama 2.0954757928848267e-09 ------ True
GPT-4o Gemini 0.015625 ------ False
GPT-4o Claude3 0.21875 ------ False
Llama Gemini 6.16488978266716e-06 ------ True
Llama Claude3 4.176981747150421e-07 ------ True
Gemini Claude3 0.453125 ------ False
----------------- justice -----------------
GPT-3.5 GPT-4o 1.5236437320709229e-05 ------ True
GPT-3.5 Llama 0.39152830513194203 ------ False
GPT-3.5 Gemini 0.22948101302608848 ------ False
GPT-3.5 Claude3 0.00017999112606048584 ------ True
GPT-4o Llama 7.62939453125e-05 ------ True
GPT-4o Gemini 1.233129296451807e-07 ------ True
GPT-4o Claude3 0.5078125 ------ False
Llama Gemini 0.028816719655878845 ------ False
Llama Claude3 0.000518798828125 ------ True
Gemini Claude3 7.660128176212311e-07 ------ True
--